In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import scipy 
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import torch
import random
import anndata
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from datasets import Loader, apply_noise
from model import scDRMAE
from evaluate import evaluate
from util import AverageMeter

In [2]:
def read_dataset1( File1 = None, File2 = None, File3 = None, File4 = None, transpose = True, test_size_prop = None, state = 0,
                  format_rna = None, formar_epi = None ):
    # read single-cell multi-omics data together

    ### raw reads count of scRNA-seq data
    adata = adata1 = None

    if File1 is not None:
        if format_rna == "table":
            adata  = sc.read(File1)
        else: # 10X format
            adata  = sc.read_mtx(File1)

        if transpose:
            adata  = adata.transpose()

    ##$ the binarization data for scEpigenomics file
    if File2 is not None:
        if formar_epi == "table":
            adata1  = sc.read( File2 )
        else  :# 10X format
            adata1  = sc.read_mtx(File2)

        if transpose:
            adata1  = adata1.transpose()

    ### File3 and File4 for cell group information of scRNA-seq and scEpigenomics data
    label_ground_truth = []
    label_ground_truth1 = []

    if state == 0 :
        if File3 is not None:
            Data2  = pd.read_csv( File3, header=0, index_col=0 )
            label_ground_truth =  Data2['trueType_y'].values

        else:
            label_ground_truth =  np.ones( len( adata.obs_names ) )

        if File4 is not None:
            Data2 = pd.read_csv( File4, header=0, index_col=0 )
            label_ground_truth1 = Data2['trueType_y'].values

        else:
            label_ground_truth1 =  np.ones( len( adata.obs_names ) )

    elif state == 1:
        if File3 is not None:
            Data2 = pd.read_table( File3, header=0, index_col=0 )
            label_ground_truth = Data2['cell_line'].values
        else:
            label_ground_truth =  np.ones( len( adata.obs_names ) )

        if File4 is not None:
            Data2 = pd.read_table( File4, header=0, index_col=0 )
            label_ground_truth1 = Data2['cell_line'].values
        else:
            label_ground_truth1 =  np.ones( len( adata.obs_names ) )

    elif state == 3:
        if File3 is not None:
            Data2 = pd.read_table( File3, header=0, index_col=0 )
            label_ground_truth = Data2['Group'].values
        else:
            label_ground_truth =  np.ones( len( adata.obs_names ) )

        if File4 is not None:
            Data2 = pd.read_table( File4, header=0, index_col=0 )
            label_ground_truth1 = Data2['Group'].values
        else:
            label_ground_truth1 =  np.ones( len( adata.obs_names ) )

    else:
        if File3 is not None:
            Data2 = pd.read_table( File3, header=0, index_col=0 )
            label_ground_truth = Data2['Cluster'].values
        else:
            label_ground_truth =  np.ones( len( adata.obs_names ) )

        if File4 is not None:
            Data2 = pd.read_table( File4, header=0, index_col=0 )
            label_ground_truth1 = Data2['Cluster'].values
        else:
            label_ground_truth1 =  np.ones( len( adata.obs_names ) )

    # split datasets into training and testing sets
    if test_size_prop > 0 :
        train_idx, test_idx = train_test_split(np.arange(adata.n_obs),
                                               test_size = test_size_prop,
                                               random_state = 200)
        spl = pd.Series(['train'] * adata.n_obs)
        spl.iloc[test_idx]  = 'test'
        adata.obs['split']  = spl.values

        if File2 is not None:
            adata1.obs['split'] = spl.values
    else:
        train_idx, test_idx = list(range( adata.n_obs )), list(range( adata.n_obs ))
        spl = pd.Series(['train'] * adata.n_obs)
        adata.obs['split']       = spl.values

        if File2 is not None:
            adata1.obs['split']  = spl.values

    adata.obs['split'] = adata.obs['split'].astype('category')
    adata.obs['Group'] = label_ground_truth
    adata.obs['Group'] = adata.obs['Group'].astype('category')

    if File2 is not None:
        adata1.obs['split'] = adata1.obs['split'].astype('category')
        adata1.obs['Group'] = label_ground_truth
        adata1.obs['Group'] = adata1.obs['Group'].astype('category')

    return adata, adata1, train_idx, test_idx, label_ground_truth, label_ground_truth1


In [3]:
def TFIDF(count_mat): 
    """
    TF-IDF transformation for matrix.

    Parameters
    ----------
    count_mat
        numpy matrix with cells as rows and peak as columns, cell * peak.

    Returns
    ----------
    tfidf_mat
        matrix after TF-IDF transformation.

    divide_title
        matrix divided in TF-IDF transformation process, would be used in "inverse_TFIDF".

    multiply_title
        matrix multiplied in TF-IDF transformation process, would be used in "inverse_TFIDF".

    """

    count_mat = count_mat.T
    divide_title = np.tile(np.sum(count_mat,axis=0), (count_mat.shape[0],1))
    nfreqs = 1.0 * count_mat / divide_title
    multiply_title = np.tile(np.log(1 + 1.0 * count_mat.shape[1] / np.sum(count_mat,axis=1)).reshape(-1,1), (1,count_mat.shape[1]))
    tfidf_mat = scipy.sparse.csr_matrix(np.multiply(nfreqs, multiply_title)).T
    return tfidf_mat, divide_title, multiply_title

In [4]:
import re
def normalize(adata, filter_min_counts=True, size_factors=True, normalize_input=True, logtrans_input=True,
              filter_gene=0.01,datatype='RNA'):
    ##过滤掉在低于1%的细胞中表达的特征
    if filter_gene:
       adata = adata[:, (adata.X > 0).sum(0) >= adata.shape[0]*0.01] 
       
    if size_factors or normalize_input or logtrans_input:
        adata.raw = adata.copy()
    else:
        adata.raw = adata
    if datatype=='ATAC':
        count_mat = adata.X.copy()
        adata.X, divide_title, multiply_title = TFIDF(count_mat)
        max_temp = np.max(adata.X)
        adata.X = adata.X / max_temp
        # return adata, divide_title, multiply_title
    if size_factors:
        sc.pp.normalize_per_cell(adata)
        adata.obs['size_factors'] = adata.obs.n_counts / np.median(adata.obs.n_counts)
    else:
        adata.obs['size_factors'] = 1.0

    if logtrans_input:
        sc.pp.log1p(adata)

    if normalize_input:
        sc.pp.scale(adata)
    return adata

In [5]:
def make_dir(directory_path, new_folder_name):
    """Creates an expected directory if it does not exist"""
    directory_path = os.path.join(directory_path, new_folder_name)
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    return directory_path


def inference(net, data_loader_test):
    net.eval()
    feature_vector = []
    labels_vector = []
    with torch.no_grad():
        for step, (x,x1, y) in enumerate(data_loader_test):
            feature_vector.extend(net.feature(x.to(device).float(), x1.to(device).float()).detach().cpu().numpy())
            labels_vector.extend(y.numpy())
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    return feature_vector, labels_vector
def chabu1(net, data_loader_test):
    net.eval()
    feature_vector = []
    labels_vector = []
    with torch.no_grad():
        for step, (x,x1, y) in enumerate(data_loader_test):
            feature_vector.extend(net.chabu(x.to(device).float(), x1.to(device).float())[0].detach().cpu().numpy())
            labels_vector.extend(y.numpy())
    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    return feature_vector, labels_vector

In [6]:
def res_search_fixed_clus(adata, fixed_clus_count, increment=0.02):
    '''
        arg1(adata)[AnnData matrix]
        arg2(fixed_clus_count)[int]

        return:
            resolution[int]
    '''
    dis = []
    resolutions = sorted(list(np.arange(0.01, 2.5, increment)), reverse=True)
    i = 0
    res_new = []
    for res in resolutions:
        sc.tl.leiden(adata, random_state=0, resolution=res)
        count_unique_leiden = len(pd.DataFrame(
            adata.obs['leiden']).leiden.unique())
        dis.append(abs(count_unique_leiden-fixed_clus_count))
        res_new.append(res)
        if count_unique_leiden == fixed_clus_count:
            break
    reso = resolutions[np.argmin(dis)]

    return reso

In [7]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(seed=10)

In [1]:
args = {}
args['batch_size'] = 256
args['n_classes'] = 4
args['dataset'] = 'InHouse'
if args['dataset'] in ['InHouse']:
    args['epochs'] = 20
    args["learning_rate"] = 0.001
elif args['dataset'] in ['human cell line mixture']:
    args['epochs'] = 50
    args["learning_rate"] = 0.001
elif args['dataset'] in ['human cell line mixture']:
    args['epochs'] = 20
    args["learning_rate"] = 0.002
else:
    args['epochs'] = 100
    args["learning_rate"] = 0.002

In [9]:
data_root = '/home/hfzhang/workplace/scMCs/scMCs/data'
X1 = os.path.join('/home/hfzhang/workplace/scDRMAE/data/GSM4476364_RNA_raw.csv.gz')
X2 = os.path.join('/home/hfzhang/workplace/scDRMAE/data/GSM4476364_ADT_raw.csv.gz')
x3 = os.path.join('/home/hfzhang/workplace/scDRMAE/data/truth_InHouse.csv') # cell type information

In [10]:
x1, x2, train_index, test_index, label_ground_truth, _ = read_dataset1(File1=X1, File2=X2,
                                                                          File3=x3, File4=None,
                                                                          transpose=True, test_size_prop=0.0,
                                                                          state=0, format_rna="table",
                                                                          formar_epi="table")

/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [11]:
x1.obs['cell_type']=label_ground_truth
x2.obs['cell_type']=label_ground_truth
x1.obs['cell_type'] = x1.obs['cell_type'].astype('category')
x2.obs['cell_type'] = x2.obs['cell_type'].astype('category')

In [12]:
classes, labels = np.unique(label_ground_truth, return_inverse=True)
classes = classes.tolist()
args['n_classes'] = len(set(classes))

In [13]:
x1 = x1[:, (x1.X > 0).sum(0) >= x1.shape[0]*0.01] 
x2 = x2[:, (x2.X > 0).sum(0) >= x2.shape[0]*0.01] 

In [14]:
# 归一化，使得不同细胞样本间可比
sc.pp.normalize_total(x1, target_sum=1e4)
sc.pp.log1p(x1)
sc.pp.highly_variable_genes(x1, n_top_genes=3000)
x_scRNA = x1[:, x1.var['highly_variable']]

/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [15]:
# 归一化，使得不同细胞样本间可比
sc.pp.normalize_total(x2, target_sum=1e4)
sc.pp.log1p(x2)
sc.pp.highly_variable_genes(x2, n_top_genes=3000)
x_scATAC = x2[:, x2.var['highly_variable']]

/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [16]:
x_scRNA=normalize(x_scRNA, filter_min_counts=False, size_factors=False, normalize_input=True, logtrans_input=False,
               filter_gene=0)

In [17]:
x_scATAC=normalize(x_scATAC,filter_gene=0, filter_min_counts=False, size_factors=False, normalize_input=True, logtrans_input=False,datatype='AAC')

In [18]:
label_ground_truth=x_scRNA.obs['cell_type'].values
classes, labels = np.unique(label_ground_truth, return_inverse=True)
classes = classes.tolist()

In [19]:
args['n_classes']

6

In [20]:
class PBMC():
    def __init__(self, rna, atac, cell_type):
        self.rna_data = rna  # 读取scRNA-seq数据
        self.atac_data = atac  # 读取scATAC-seq数据
        self.cell_type = cell_type  # 读取细胞类型标签

    def __len__(self):
        return len(self.rna_data)  # 返回数据集的长度

    def __getitem__(self, idx):
        rna_sample = self.rna_data[idx]  # 获取scRNA-seq样本
        atac_sample = self.atac_data[idx]  # 获取scATAC-seq样本
        cell_type_label = self.cell_type[idx]  # 获取细胞类型标签

        return rna_sample, atac_sample, cell_type_label

In [21]:
device='cuda:0'

In [24]:
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
import anndata
crossFold=5
kf = KFold(n_splits=crossFold, shuffle=True, random_state=42)
nmis_test=[]
aris_test=[]
amis_test=[]
for fold, (train_indices, val_indices) in enumerate(kf.split(x_scRNA)):
    con_loss=[]
    re_loss=[]
    results=[]
    x_scRNA_train, x_scRNA_test = x_scRNA[train_indices], x_scRNA[val_indices]
    x_scATAC_train,x_scATAC_test =x_scATAC[train_indices], x_scATAC[val_indices]
    x_scRNA_train_aug, x_scATAC_train_aug=x_scRNA_train, x_scATAC_train

    y_train=labels[train_indices]
    y_test=labels[val_indices]

    x_scRNA_train=torch.from_numpy(x_scRNA_train_aug.X.toarray())
    x_scATAC_train=torch.from_numpy(x_scATAC_train_aug.X.toarray())
    x_scRNA_test=torch.from_numpy(x_scRNA_test.X.toarray())
    x_scATAC_test=torch.from_numpy(x_scATAC_test.X.toarray())
    x_scATAC_all=torch.from_numpy(x_scATAC.X)
    x_scRNA_all=torch.from_numpy(x_scRNA.X)

    N1_train, M1_train = np.shape(x_scRNA_train)
    N2_train, M2_train = np.shape(x_scATAC_train)
    N1_test, M1_test = np.shape(x_scRNA_test)
    N2_test, M2_test = np.shape(x_scATAC_test)
    ##训练
    dataset_train = PBMC(x_scRNA_train,x_scATAC_train,y_train)
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=256,
        shuffle=True,
        drop_last=True,
    )
    dataset = PBMC(x_scRNA_all,x_scATAC_all,labels)
    data_loader_val = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=256,
        shuffle=False,
        drop_last=False,
    )
    data_loader_all = torch.utils.data.DataLoader(
        dataset,
        batch_size=256,
        shuffle=False,
        drop_last=False,
    )
    dataset_test = PBMC(x_scRNA_test,x_scATAC_test,y_test)
    data_loader_test = torch.utils.data.DataLoader(
        dataset_test,
        batch_size=256,
        shuffle=False,
        drop_last=False,
    )
    dims = [M1_train, M2_train]
    view = 2
    data_size = len(label_ground_truth)
    data_size_train = len(y_train)
    data_size_test = len(y_test)
    class_num = len(classes)
    init_lr = args["learning_rate"]
    count=0
    max_epochs = args['epochs']
    mask_probas = [0.4]*M1_train
    mask_probas1 = [0.4]*M2_train
    model = scDRMAE(
                num_genes=M1_train,
                num_ATAC=M2_train,
                hidden_size=128,
                masked_data_weight=0.75,
                mask_loss_weight=0.7
            ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)

    if not os.path.exists('./models'):
        os.makedirs('./models')
    best_ari=0
    best_nmi=0
    best_epoch=0
    for epoch in range(max_epochs):
        model.train()
        meter = AverageMeter()
        for i, (x,x1, y) in enumerate(data_loader_train):
            x=x.float()
            x1=x1.float()
            x = x.to(device)
            x1 = x1.to(device)
            x_corrputed, mask = apply_noise(x, mask_probas)
            x_corrputed1, mask1 = apply_noise(x1, mask_probas1)
            optimizer.zero_grad()
            x_corrputed_latent, loss = model.loss_mask(x_corrputed, x, mask,x_corrputed1, x1, mask1,epoch,max_epochs)
            loss.backward()
            optimizer.step()
            meter.update(loss.detach().cpu().numpy())

        latent, true_label = inference(model, data_loader_val)
        clustering_model = KMeans(n_clusters=args["n_classes"],n_init=6)
        clustering_model.fit(latent)
        pred_label = clustering_model.labels_
            
        nmi, ari, acc,ami = evaluate(true_label, pred_label)
        ss = silhouette_score(latent, pred_label)
        if ari>best_ari:
            best_ari=ari
            best_epoch=epoch
            best_state=model.state_dict()


    print('best_epoch',best_epoch)
    print('best_ari',best_ari)
    model.load_state_dict(best_state)
    with torch.no_grad():
        model.eval()
        
        latent, true_label = inference(model,
                                        data_loader_test)
        clustering_model = KMeans(n_clusters=args["n_classes"])
        clustering_model.fit(latent)
        pred_label = clustering_model.labels_
        nmi_test, ari_test, acc_test, ami_test = evaluate(true_label, pred_label)
        print("\ttest: [nmi: %f] [ari: %f] [ami: %f]" % (nmi_test, ari_test, ami_test))

    aris_test.append(ari_test)
    nmis_test.append(nmi_test)
    amis_test.append(ami_test)
nmi=sum(nmis_test)/(fold+1)
ari=sum(aris_test)/(fold+1)
ami=sum(amis_test)/(fold+1)
print('nmi为{},ari为{},ami{}'.format(nmi,ari,ami))


best_epoch 8
best_ari 0.9476969307916453


/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


	test: [nmi: 0.951458] [ari: 0.975006] [ami: 0.949422]
best_epoch 18
best_ari 0.9592236690668591


/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


	test: [nmi: 0.940318] [ari: 0.935737] [ami: 0.938043]
best_epoch 7
best_ari 0.9614096892955325


...

	test: [nmi: 0.933087] [ari: 0.921679] [ami: 0.930495]
best_epoch 8
best_ari 0.9502687482296138


/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


	test: [nmi: 0.956394] [ari: 0.969171] [ami: 0.954715]
best_epoch 15
best_ari 0.9532722971185495


/home/hfzhang/software/anaconda3/envs/scEMC/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


	test: [nmi: 0.945497] [ari: 0.962437] [ami: 0.943286]
nmi为0.9453506924231665,ari为0.9528059443926515,ami0.9431922043078863
